In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import pandas as pd

import gmaps
import gmaps.geojson_geometries
from geographiclib.geodesic import Geodesic
import googlemaps

import hubspot

In [2]:
gmaps.configure(api_key=os.environ["GMAPS_API_KEY"])
client = hubspot.Client.create(access_token=os.environ["HUBSPOT_API_KEY"])
geocode = googlemaps.Client(key=os.environ["GEOCODE_API_KEY"])

In [3]:
def pull_properties(hubspot_raw_df):
    # Extract all unique property keys from 'properties' dictionary
    property_keys = set().union(*hubspot_raw_df['properties'].apply(lambda x: x.keys()))

    # Create new columns for each property key
    for key in property_keys:
        hubspot_raw_df[key] = ''

    # Extract values for each property key and update the corresponding cell in the dataframe
    for i, row in hubspot_raw_df.iterrows():
        for key, value in row['properties'].items():
            if value is None:
                value = ''  # replace None values with empty strings
            hubspot_raw_df.at[i, key] = value

    # Drop 'properties' column and return the updated dataframe
    return hubspot_raw_df.drop('properties', axis=1)

In [4]:
### HUBSPOT TABLE ###

company_list = []
has_more = True
after_value = 0
while has_more:
    api_response = client.crm.companies.basic_api.get_page(limit=100, after = after_value, properties=["name", "send_to_cs", "record_type", "address", "city", "full_address"], archived=False).to_dict()
    company_list.extend(api_response['results'])
    try:
        after_value = api_response['paging']['next']['after']
    except:
        has_more = False
print('Companies included: ' + str(len(company_list)))
company_ids = (pd.DataFrame(company_list)['id'])
company_list_df = pd.DataFrame(company_list)

hubspot_clients = pull_properties(company_list_df).query('send_to_cs == "true"').rename({'id':'hubspotID'}, axis=1)
hubspot_clients = hubspot_clients.reset_index(drop=True)

Companies included: 13917


In [5]:
geocode.geocode(hubspot_clients['full_address'][0])

[{'address_components': [{'long_name': 'Suite 103',
    'short_name': 'Suite 103',
    'types': ['subpremise']},
   {'long_name': '500', 'short_name': '500', 'types': ['street_number']},
   {'long_name': 'Carson Plaza Drive',
    'short_name': 'Carson Plaza Dr',
    'types': ['route']},
   {'long_name': 'Carson',
    'short_name': 'Carson',
    'types': ['locality', 'political']},
   {'long_name': 'Los Angeles County',
    'short_name': 'Los Angeles County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'California',
    'short_name': 'CA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '90746', 'short_name': '90746', 'types': ['postal_code']}],
  'formatted_address': '500 Carson Plaza Dr Suite 103, Carson, CA 90746, USA',
  'geometry': {'location': {'lat': 33.8447059, 'lng': -118.2659098},
   'location_type': 'ROOFTOP',
   'view

In [6]:
lat_lng = geocode.geocode(hubspot_clients['full_address'][0])[0]['geometry']['location']

In [15]:
location = (33.8447059, -118.2659098) #lat_lng.values()
fig = gmaps.figure(center=location, zoom_level=11)
fig

Figure(layout=FigureLayout(height='420px'))